In [2]:
# AutoGluon - Comparación de modelos con distintos LAGs para predicción febrero 2020

In [3]:
## 1. Importar librerías
import pandas as pd
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame

In [4]:
## 2. Cargar datasets
df_sellin = pd.read_csv("sell-in.txt", sep="\t")
df_productos = pd.read_csv("tb_productos.txt", sep="\t")
with open("product_id_apredecir201912.TXT", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

In [5]:
## 3. Preprocesamiento
df_sellin['timestamp'] = pd.to_datetime(df_sellin['periodo'], format='%Y%m')
df_filtered = df_sellin[
    (df_sellin['timestamp'] <= '2019-12-01') &
    (df_sellin['product_id'].isin(product_ids))
]
df_grouped = df_filtered.groupby(['timestamp', 'customer_id', 'product_id'], as_index=False)['tn'].sum()
df_monthly_product = df_grouped.groupby(['timestamp', 'product_id'], as_index=False)['tn'].sum()
df_monthly_product['item_id'] = df_monthly_product['product_id']



In [6]:
# --- Agregar lags manualmente ---
df_lags = df_monthly_product.copy()
df_lags = df_lags.sort_values(by=['product_id', 'timestamp'])

# Agregar 6 lags
for i in range(1, 7):
    df_lags[f'tn_lag_{i}'] = df_lags.groupby('product_id')['tn'].shift(i)

# Eliminar filas con NaNs (por ejemplo, las primeras 6 de cada producto)
df_lags = df_lags.dropna().reset_index(drop=True)


In [8]:

# Rename 'tn' to 'target' as required by AutoGluon



df_lags_renamed = df_lags.rename(columns={'tn': 'target'})
ts_data = TimeSeriesDataFrame.from_data_frame(
    df_lags_renamed,
    id_column='item_id',
    timestamp_column='timestamp'
)
ts_data = ts_data.fill_missing_values()

In [9]:
df_lags_renamed['item_id'] = df_lags['product_id']

In [10]:

ts_data_lags = TimeSeriesDataFrame.from_data_frame(
    df_lags_renamed,
    id_column='item_id',
    timestamp_column='timestamp'
)

ts_data_lags = ts_data_lags.fill_missing_values()

predictor_lags = TimeSeriesPredictor(
    prediction_length=1,
    target='target',
    freq='MS',
    path='modelo_con_lags_manual/'
)

predictor_lags.fit(ts_data_lags)


Beginning AutoGluon training...
AutoGluon will save models to '/Users/patricialorenasarmientotagle/austral-labo-iii/notebooks/modelo_con_lags_manual'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.9.6
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.5.0: Tue Apr 22 19:53:27 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T6041
CPU Count:          12
GPU Count:          0
Memory Avail:       8.59 GB / 24.00 GB (35.8%)
Disk Space Avail:   275.54 GB / 460.43 GB (59.8%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'MS',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 1,
 'prediction_length': 1,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'target',
 'verbosity': 2}

train_data with frequ

In [11]:
# --- Predecir febrero 2020 ---
forecast = predictor_lags.predict(ts_data_lags)

# --- Extraer predicción media por producto ---
forecast_feb2020 = forecast['mean'].reset_index()[['item_id', 'mean']]
forecast_feb2020.columns = ['product_id', 'tn']

# --- Exportar resultados a CSV ---
forecast_feb2020.to_csv("predicciones_febrero2020_lags_manual.csv", index=False)

# --- Mostrar primeros resultados ---
forecast_feb2020.head()


data with frequency 'IRREG' has been resampled to frequency 'MS'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


,product_id,tn
0,20001,1312.212270
1,20002,1066.463859
2,20003,741.460424
3,20004,558.979393
4,20005,507.088888


In [12]:
df_predicciones = pd.read_csv("predicciones_febrero2020_lags_manual.csv")


In [13]:
with open("product_id_apredecir201912.TXT", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

df_objetivo = pd.DataFrame({"product_id": product_ids})


In [15]:
# Hacer merge para ver cuáles faltan
df_merged = df_objetivo.merge(df_predicciones, on="product_id", how="left", indicator=True)

# Filtrar los que no fueron predichos
faltantes = df_merged[df_merged["_merge"] == "left_only"]["product_id"].tolist()

print(f"❗ Total de productos faltantes: {len(faltantes)}")
print("Algunos ejemplos:", faltantes[:46])


❗ Total de productos faltantes: 46
Algunos ejemplos: [20127, 20210, 20213, 20236, 20257, 20261, 20440, 20615, 20621, 20662, 20673, 20686, 20694, 20703, 20711, 20720, 20762, 20815, 20845, 20886, 20899, 20904, 20910, 20912, 20933, 20942, 20953, 20962, 20968, 20975, 20995, 21006, 21035, 21039, 21058, 21079, 21087, 21097, 21109, 21110, 21112, 21129, 21144, 21146, 21154, 21214]


In [16]:
import pandas as pd

# --- 1. Cargar predicciones obtenidas con lags ---
df_predicciones = pd.read_csv("predicciones_febrero2020_lags_manual.csv")

# --- 2. Cargar lista completa de product_id a predecir ---
with open("product_id_apredecir201912.TXT", "r") as f:
    product_ids = [int(line.strip()) for line in f if line.strip().isdigit()]

df_objetivo = pd.DataFrame({"product_id": product_ids})

# --- 3. Detectar los productos que faltan ---
df_merged = df_objetivo.merge(df_predicciones, on="product_id", how="left", indicator=True)
faltantes = df_merged[df_merged["_merge"] == "left_only"]["product_id"].tolist()

print(f"❗ Faltan predicciones para {len(faltantes)} productos.")

# --- 4. Calcular el promedio de ventas de 2019 por producto ---
df_sellin["periodo"] = df_sellin["periodo"].astype(str)
df_filtrado_2019 = df_sellin[
    (df_sellin["product_id"].isin(faltantes)) &
    (df_sellin["periodo"].between("201901", "201912"))
]

df_promedios = (
    df_filtrado_2019.groupby("product_id")["tn"]
    .sum()
    .div(12)  # promedio mensual
    .reset_index()
    .rename(columns={"tn": "tn"})
)

print(f"✅ Calculados promedios para {df_promedios.shape[0]} productos.")

# --- 5. Unir ambas fuentes de predicción ---
df_final = pd.concat([df_predicciones, df_promedios], axis=0).sort_values("product_id").reset_index(drop=True)

# --- 6. Guardar resultado final ---
df_final.to_csv("predicciones_febrero2020_completas.csv", index=False)

print(f"✅ Archivo final generado: predicciones_febrero2020_completas.csv ({df_final.shape[0]} productos)")


❗ Faltan predicciones para 46 productos.
✅ Calculados promedios para 46 productos.
✅ Archivo final generado: predicciones_febrero2020_completas.csv (780 productos)


In [17]:
import pandas as pd

# --- 1. Cargar predicciones finales y promedio previo ---
df_final = pd.read_csv("predicciones_febrero2020_completas.csv")  # tn por producto_id
df_promedio_simple = pd.read_csv("promedio_12m_febrero2020_2.csv")  # tn por producto_id

# --- 2. Renombrar columnas para claridad ---
df_final = df_final.rename(columns={"tn": "tn_modelo"})
df_promedio_simple = df_promedio_simple.rename(columns={"tn": "tn_promedio"})

# --- 3. Combinar ambos por product_id ---
df_ensemble = df_final.merge(df_promedio_simple, on="product_id", how="inner")

# --- 4. Calcular promedio simple (ensemble) ---
df_ensemble["tn_ensemble"] = (
    df_ensemble["tn_modelo"] + df_ensemble["tn_promedio"]
) / 2

# --- 5. Guardar resultado final ---
df_ensemble[["product_id", "tn_ensemble"]].to_csv("predicciones_febrero2020_ensemble_final.csv", index=False)

print(f"✅ Ensemble generado para {df_ensemble.shape[0]} productos.")


✅ Ensemble generado para 780 productos.
